In [1]:
#Choosing the GPU to use
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, for our server either "0" , "1", "2" or "3";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [2]:
%%capture

import warnings
import sys
import shutil
import math
import json
import gc

import numpy as np
from keras.applications import ResNet50V2, Xception, VGG19, DenseNet121
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import backend 

#include the directory with custom functions
sys.path.insert(1, '../custom_functions')
from train_functions import *
from image_generators import get_generator
from custom_callbacks import save_pkl, load_pkl
from custom_callbacks import modelTracker
from custom_callbacks import saveModelCallback
from custom_callbacks import EveryEpochSaver

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

warnings.simplefilter('ignore',  DeprecationWarning)

In [3]:
#Verifying the version of tensorflow
from __future__ import absolute_import, division, print_function
import tensorflow as tf
print(tf.__version__)

1.14.0


In [4]:
#Opening the config file
TRAIN_CONFIG_PATH = "train_config.json"
with open(TRAIN_CONFIG_PATH, "r") as read_file:
    train_config = json.load(read_file)
train_config

{'dataset_dir': '../../datasets/dataset_mixed/train_flipped/',
 'model_name': 'cv_prod_test',
 'all_models_dir': '../../models/',
 'pretrained_model_path': '',
 'cross_validation': {'dataframes_dir': '../../datasets/dataset_mixed/crossval_10',
  'used': False,
  'folds': 1},
 'dataframe_path': '../../datasets/dataset_mixed/train_flipped/df_Forgetful_Fabio.csv',
 'training_is_finised': False,
 'image_size': 224,
 'opt_dict': {'optimizer_type': 'SGD',
  'momentum': 0.002,
  'learning_rate': 0.005,
  'nesterov': True},
 'activation': 'sigmoid',
 'use_weights': False,
 'non_retinas_weight_factor': 1.0,
 'reduce_lr_factor': 0.5,
 'train_val_ratio': 0.25,
 'last_layer_output_dim': 512,
 'normalization_mode': 'samplewise',
 'class_mode': 'binary',
 'last_layer_dropout_rate': 0.9,
 'reduce_lr_min': 1e-09,
 'epochs': 10,
 'keras_model_name': 'resnet50v2',
 'reduce_on_plateau_patience': 4,
 'reduce_lr_patience': 6,
 'loss': 'binary_crossentropy',
 'batch_size': 32,
 'input_shape': [224, 224, 3],

In [5]:
#If you want to edit config from this notebook use this cell with train_config["key"] = "value" 
#WARNING: file "config.json" will be overwritten

#pretrainedModelType should be chosen manually
pretrainedModelType = ResNet50V2
train_config["trainable_layers"]["from"] = 100
train_config["trainable_layers"]["to"] = 197
train_config["model_name"] ="cv_prod_test"
train_config["keras_model_name"] = pretrainedModelType().name
#Turning on or turning off cros-validation
train_config["cross_validation"] = {
    "dataframes_dir": '../../datasets/dataset_mixed/crossval_10',
    "used": False,
    "folds": 1 
}
#path for already trained model. If it is an empty line, no pretrained model is used
train_config["pretrained_model_path"]=""

with open(TRAIN_CONFIG_PATH, "w+") as write_file:
    json.dump(train_config, write_file)
    
#If MODEL_OVERWRITE_MODE is "on" than the model will be wirtten in the same dir at each run.
#If MODEL_OVERWRITE_MODE is "off" the exception will be accerted in the next cell execution 
MODEL_OVERWRITE_MODE = "on"
#This line creates variabels with the same names as train_config.keys() and values from train_config
locals().update(train_config)

In [6]:
#Adding date, time and computer name to the config
model_config = make_model_config(train_config)

#Creating the model dir inside train_config["all_models_dir"] and several subdirs.
model_dir, temp_dataframes_dirs, history_dirs, model_weights_dirs = make_place_for_the_model(model_config,
                                                                       MODEL_OVERWRITE_MODE
                                                                      )


#Defining dataframes.
#For cross-validation the dataframes should be prepared in advance and be storeed in 
#train_config["cross_validation"]["dataframes_dir"]
#Otherwise train dataframe is randomly spliteed into train and val

if train_config["cross_validation"]["used"]:
    dfs_train, dfs_val = import_kfold_dataframes(train_config)
else:
    dataframe_train, dataframe_val = create_temp_dfs(dataframe_path,
                                                                temp_dataframes_dirs[0], 
                                                                model_name,
                                                                train_val_ratio
                                                             )
    dfs_train = [dataframe_train]
    dfs_val = [dataframe_val]


#Defining the normaliztion parameters for generators
samplewise_std_normalization, samplewise_center,\
featurewise_std_normalization, featurewise_center,\
rescale = normalization_mode_setter(normalization_mode)
        

In [7]:
%%capture
#from keras.utils import plot_model
#plot_model(pretrainedModelType(), to_file='model_graphs/DenseNet121.png')

In [8]:
#Importing the pretrained model. If pretrained_model_path is empty,
# the one from Keras library is used, with several layers being appended
model = make_model(train_config, pretrainedModelType)
for n, layer in enumerate(model.layers):
    print(" %3d) %-30s   trainable: %s" % (n,str(layer.name), layer.trainable))

   0) input_2                          trainable: False
   1) conv1_pad                        trainable: False
   2) conv1_conv                       trainable: False
   3) pool1_pad                        trainable: False
   4) pool1_pool                       trainable: False
   5) conv2_block1_preact_bn           trainable: False
   6) conv2_block1_preact_relu         trainable: False
   7) conv2_block1_1_conv              trainable: False
   8) conv2_block1_1_bn                trainable: False
   9) conv2_block1_1_relu              trainable: False
  10) conv2_block1_2_pad               trainable: False
  11) conv2_block1_2_conv              trainable: False
  12) conv2_block1_2_bn                trainable: False
  13) conv2_block1_2_relu              trainable: False
  14) conv2_block1_0_conv              trainable: False
  15) conv2_block1_3_conv              trainable: False
  16) conv2_block1_out                 trainable: False
  17) conv2_block2_preact_bn           trainable

In [9]:
#Number of trainable parameters
trainable_count = int(
    np.sum([backend.count_params(p) for p in set(model.trainable_weights)]))
non_trainable_count = int(
    np.sum([backend.count_params(p) for p in set(model.non_trainable_weights)]))
print('{:,}'.format(trainable_count) )

21,610,497


In [10]:
#Training procedure

with open(os.path.join(model_dir,"model_config.json"), "w+") as write_file:
    json.dump(model_config, write_file)
     
if train_config["cross_validation"]["used"]:
    number_of_cycles = cross_validation["folds"]
else:
    number_of_cycles = 1
    
for k in range(cross_validation["folds"]):
    
    print("Cross-validation itteration number", k)
    
    #Defining the dir where to save currnet progress
    if train_config["cross_validation"]["used"]:
        split_dir = os.path.join(model_dir, "_".join(["split", str(k)]))
        if not os.path.exists(split_dir):
            raise Exception("No split path!")
    else:
        split_dir = model_dir
        
    #Selecting dataframes  
    df_train = dfs_train[k] 
    df_val = dfs_val[k] 
   
    print("\nCreating train generator")
    
    #Defining train and val generators
    train_generator = get_generator(train_config, df_train, shuffle = True,  augmentation = True)
    train_class_weights = get_class_weigts(train_generator)
    train_class_weights[train_generator.class_indices["non-retina"]] *= non_retinas_weight_factor 
    print('Train class_indices: ', train_generator.class_indices)
    print('train_class_weights: ', train_class_weights)
    if not use_weights:
        train_class_weights = {
            1: 1.,
            0: 1.
        }
    print('train_class_weights to use: ', train_class_weights)
    print("\nCreating val generator")
    val_generator = get_generator(train_config, df_val , shuffle = False,  augmentation = True)
    print('Val class_indices: ', val_generator.class_indices)
    
    print("\nGenerators for metrics computing")
        
    #Defining the model 
    model = make_model(train_config, pretrainedModelType)
    
    #Forming callback list
    #Note, that to compute the metric two additional generators required (without augmentation)
    metrics_train_generator = get_generator(train_config, df_train, shuffle=False, augmentation=True)
    metrics_val_generator = get_generator(train_config, df_val, shuffle=False, augmentation=False)
    model_tracker = modelTracker(history_dirs[k],
                             metrics_train_generator,
                             metrics_val_generator,
                             model_name,
                             split_dir,
                             monitor = "val_ROCAUC"
                            )
    EpochSaver = EveryEpochSaver(model_name, model_weights_dirs[k])
    last_model_path = os.path.join(split_dir,"last_" + model_name + '.h5')
    best_val_acc_model_path = os.path.join(split_dir,"keras_best_val_acc_" + model_name + '.h5') 
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=reduce_lr_patience, 
                                            verbose=1, 
                                            factor=reduce_lr_factor, 
                                            min_lr=reduce_lr_min)   
    callbacks_list = [learning_rate_reduction, model_tracker, EpochSaver]
    
       
    #Defining the steps per each epoch
    train_samples_number = df_train.shape[0]
    steps_per_epoch = math.ceil(train_samples_number // batch_size)
    
    val_samples_number = df_val.shape[0]
    val_steps = math.ceil(val_samples_number // batch_size)

    #Starting the training
    print("\nModel \"%s\" is being trained."% (model_name))
    print("Class mode is \"%s\"."% train_generator.class_mode)
    print("The model is contained in the directory: %s \n" % split_dir)
    history = model.fit_generator( train_generator,
                                steps_per_epoch= steps_per_epoch,
                                validation_data=val_generator,
                                validation_steps=val_steps,
                                callbacks=callbacks_list,
                                epochs=epochs,
                                class_weight=train_class_weights,
                                verbose=3)
    
    #Saving last model and the history
    model.save(last_model_path)
    save_pkl(history.history, os.path.join(history_dirs[k], 'history_Keras.pkl')) 
    
    print("\n************************************************************************ \n")
    
model_config_path = os.path.join(all_models_dir, model_name, "_".join([model_name, "config.json"]))
model_config["training_is_finished"] = True
with open(model_config_path, "w+") as write_file:
    json.dump(model_config,write_file)

Cross-validation itteration number 0

Creating train generator
Found 1083 validated image filenames belonging to 2 classes.
Train class_indices:  {'non-retina': 0, 'retina': 1}
train_class_weights:  {1: 1.0, 0: 0.9}
train_class_weights to use:  {1: 1.0, 0: 1.0}

Creating val generator
Found 362 validated image filenames belonging to 2 classes.
Val class_indices:  {'non-retina': 0, 'retina': 1}

Generators for metrics computing
Found 1083 validated image filenames belonging to 2 classes.
Found 362 validated image filenames belonging to 2 classes.

Model "cv_prod_test" is being trained.
Class mode is "binary".
The model is contained in the directory: ../../models/cv_prod_test 

Epoch 1/10
Train--Val loss: 0.705298 -- 0.704469, Train--Val ROCAUC: 0.518077 -- 0.518049
Epoch 2/10
Model with best val_ROCAUC is saved
Train--Val loss: 0.688351 -- 0.689529, Train--Val ROCAUC: 0.557303 -- 0.563853
Epoch 3/10
Model with best val_ROCAUC is saved
Train--Val loss: 0.673731 -- 0.673685, Train--Val RO